In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
# TODO: check tuned vector angles range: -pi to pi (in exported hdf5 file, seems like it)?


In [ ]:
#import necessary libraries
import pandas as pd
import h5py
import numpy as np
from scipy.stats import zscore,kstest
from scipy.signal import find_peaks
import random
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import seaborn as sns

import sys  # for importing one level above
sys.path.append("..")
from placecode.spatial_coding_functions import *
from placecode.utils import open_file, open_dir, read_spatial
from placecode.analysis_info import ExpInfo, AnalysisParams

In [ ]:
perform_ks = True  # perform Kolmogorov-Smirnov test? It takes quite a while.
save_results = True  # flag whether to save results into hdf5 file 

## Locate files

In [ ]:
fpath_expinfo = open_file("Select experiment info json file!")
exp_info = ExpInfo(fpath_expinfo)  # TODO: mount data folder! so we can access it

In [ ]:
# open analysis parameters, append mouse/session-specific data, save it later with results.
fpath_analysis_params = open_file("Select analysis parameters json file!")
analysis_params = AnalysisParams(fpath_analysis_params)
analysis_params.read_exp_info(exp_info)  # extract necessary experiment information for analysis

In [ ]:
# make sure the necessary files exist
assert os.path.exists(exp_info.fpath_caim)
assert os.path.exists(exp_info.fpath_loco)

In [ ]:
# select folder to save results
output_folder = open_dir("Select folder for output")  
# TODO: add output folder to analysis parameters? Create/check folder mouse_id -> condition, save results there

## Load data

In [ ]:
# load CaImAn data
with h5py.File(exp_info.fpath_caim, "r") as hf_caim:
    # temporal
    temporal_raw=hf_caim['estimates']['C'][()]
    n_components, n_frames = temporal_raw.shape
    # access a single temporal component as temporal_raw[i]

    # spatial
    resolution = hf_caim["dims"][()]
    A_data = hf_caim["estimates"]["A"]["data"][()]
    A_indices = hf_caim["estimates"]["A"]["indices"][()]
    A_indptr = hf_caim["estimates"]["A"]["indptr"][()]
    A_shape = hf_caim["estimates"]["A"]["shape"][()]
    spatial = read_spatial(A_data, A_indices, A_indptr, A_shape, n_components, resolution, unflatten=True)

In [ ]:
# load loco data
# TODO: include stripes, distance per round, etc. in loco data cut to scanner time frame. Missing in Martin's code?
#   check https://github.com/mitlabence/matlab-2p/issues/11
dict_loco = dict()
with h5py.File(exp_info.fpath_loco, "r") as hf_loco:
    for dset_name in hf_loco["inferred"]["belt_scn_dict"].keys():
        dtype = np.int16 if dset_name in ["round", "rounds", "stripes"] else np.float64
        dict_loco[dset_name] = hf_loco["inferred"]["belt_scn_dict"][dset_name][()].astype(dtype)
print(dict_loco.keys())


## Preprocess data

Create z-score of temporal components

In [ ]:
temporal_z = zscore(temporal_raw, axis=1)

Create binary trace

In [ ]:
temporal_binary = np.zeros(shape=temporal_raw.shape, dtype=bool)
for i_unit in range(n_components):
    idx_peaks = find_peaks(temporal_raw[i_unit], height=analysis_params.peak_threshold, distance=analysis_params.peak_distance)[0]
    temporal_binary[i_unit][idx_peaks] = 1

### Filter rounds
Only use rounds where the total length adds up to the expected belt length, and filter by locomotion criteria as defined in Danielson et al. 2016

In [ ]:
expected_distance=exp_info.belt_length_mm
lv_rounds = dict_loco["rounds"].copy()  
lv_distPR = dict_loco["distance"].copy() # belt_scn_dict hast distance per round as distance, see issue above
lv_speed = dict_loco["speed"].copy()
n_rounds=lv_rounds.max()  # number of finished rounds
rounds = []
round_flags = np.zeros(n_rounds, dtype=np.int8)  # 1 if corresponding round included in analysis, 0 otherwise

for round in range(1,n_rounds+1):
    dist_current_round=lv_distPR[lv_rounds==round][-1]
    #print(dist_current_round)
    if abs(dist_current_round-expected_distance)<15:
        rounds.append(round-1)
        round_flags[round-1] = 1
    else:
        print(f"Not using {round}")


num_rounds=len(rounds)
print(f'Rounds (starting with 0): {rounds}\n Number of rounds used: {num_rounds}, total {n_rounds}')

# save as parameter a binary array on which round was included
analysis_params.rounds_included = round_flags

In [ ]:
# filter the data
included_rounds_frames =  np.isin(dict_loco["rounds"], np.array(rounds))  # filter to only those rounds that count (rounds 0-indexing)
loco_frames = get_running_frames(dict_loco["speed"], min_peak_speed=0.1)  # TODO: convert labview speed to cm/s somehow? Maybe need original (100 Hz) data
idx_filtered = np.logical_and(included_rounds_frames, loco_frames)

dict_loco_cut = dict()
for k in dict_loco:
    dict_loco_cut[k] = dict_loco[k][idx_filtered]
temporal_raw_cut = temporal_raw[:, idx_filtered]
temporal_z_cut = temporal_z[:, idx_filtered]
temporal_binary_cut = temporal_binary[:, idx_filtered]
rounds_cut = lv_rounds[idx_filtered]  # get corresponding round index
distPR_cut = lv_distPR[idx_filtered]
lv_speed_cut = lv_speed[idx_filtered]

included_frames = np.arange(0, n_frames)
included_frames_cut = included_frames[idx_filtered]

### Calculate signal strength map (originally firing rate map)
Defined as the average of the temporal component Z-score over one spatial bin

In [ ]:
n_bins = analysis_params.n_bins
n_units = n_components
analysis_params.n_units = n_units

In [ ]:
bin_borders = np.linspace(0, analysis_params.belt_length_mm, analysis_params.n_bins, endpoint=True)
#  get signal strength maps
ssm_raw = signal_strength_map(temporal_raw_cut, dict_loco_cut["rounds"], dict_loco_cut["distance"], n_bins)
ssm_z = signal_strength_map(temporal_z_cut, dict_loco_cut["rounds"], dict_loco_cut["distance"], n_bins)
# average over rounds
ssm_raw_avg = np.mean(ssm_raw, axis=1)
ssm_z_avg = np.mean(ssm_z, axis=1)
# create binary mask with 1 where events in ssm appear
ssm_events_mask = make_binary(ssm_z,peak_threshold=analysis_params.peak_threshold,peak_distance=analysis_params.peak_distance)

### Select "active units"
These are units that show at least `AnalysisParams.n_events_threshold` "events".


In [ ]:
n_events_threshold = analysis_params.n_events_threshold
idx_active_units = np.where(np.sum(ssm_events_mask, axis=(1, 2)) >= n_events_threshold )[0]  # sum up flags for each spatial bin in neach round
percent_active_units = len(idx_active_units)/ssm_events_mask.shape[0]
print(f"Using {percent_active_units*100:.2f}% of all units (at least {n_events_threshold} firing events)")

### Check processed data
Plot locomotion (grey) and temporal component (green). Show frames included in analysis (blue horizontal lines) and firing events that are included (vertical lines color coded (10 separate colors) based on bins)

In [ ]:
# TODO: make scaling factor global (find global max in loco and in temporal components) to easily compare loco and Ca spikes across rounds and across cells
bin_beginnings = np.linspace(0, analysis_params.belt_length_mm, analysis_params.n_bins, endpoint=False)  # the mm value of the beginning of each bin
cmap = mpl.colormaps["tab10"]
def plot_debug(i_cell):
    fig, axs = plt.subplots(2, 1, figsize=(12, 24))
    offset = 0.0
    axs[0].imshow(ssm_z[i_cell])
    axs[0].set_ylim((-1, np.max(rounds)+1))
    for round in rounds:
        idx_current_round = lv_rounds == round
        temporal_current_round = temporal_z[i_cell][idx_current_round]
        speed_current_round = lv_speed[idx_current_round]
        included_frames_current_round = idx_filtered[idx_current_round]
        dist_current_round = lv_distPR[idx_current_round]
        # get all spikes that occur during the included frames
        events_during_included_frames = np.logical_and(temporal_binary[i_cell][idx_current_round], included_frames_current_round)
        firing_current_round = np.nonzero(events_during_included_frames) 
        # get corresponding spatial bin for each firing event 
        # find first element greater than distance at which cell fired (index of next bin)
        # subtract 1 to find the bin index
        i_bins_events = np.searchsorted(bin_beginnings, dist_current_round[firing_current_round], side="right") - 1
        # get corresponding indices of color map 
        i_event_colors = i_bins_events%len(cmap.colors)
        # create locomotion segments as (loco_begin_frame, loco_end_frame)
        included_segments = []
        i_begin_segment = 0
        for i in range(1, len(included_frames_current_round)):
            # detect beginning of a segment: previous frame not included, current included
            if included_frames_current_round[i] and not included_frames_current_round[i-1]:
                i_begin_segment = i
            # detect end of a segment: previous frame included, current not included
            elif not included_frames_current_round[i] and included_frames_current_round[i-1]:
                i_end_segment = i-1
                # end of segment found: add segment to list
                included_segments.append((i_begin_segment, i_end_segment))
            # for last frame: if it is part of last segment, it has to be added to that segment.
            # If it is a new segment, this new segment of length 1 has to be added
            elif i == len(included_frames_current_round) - 1:
                if included_frames_current_round[i]:
                    if included_frames_current_round[i-1]:
                        i_end_segment = i
                        included_segments.append((i_begin_segment, i_end_segment))
                else:
                    included_segments.append((i, i))

        temp_min = np.min(temporal_current_round)
        temp_max = np.max(temporal_current_round)
        speed_min = np.min(speed_current_round)
        speed_max = np.max(speed_current_round)
        axs[1].hlines(y=[offset-0.05 for i in range(len(included_segments))], xmin=[included_segments[i][0] for i in range(len(included_segments))], xmax=[included_segments[i][1] for i in range(len(included_segments))], linewidth=3 )
        # plot the locomotion velocity
        if speed_max != speed_min:
            axs[1].plot((speed_current_round - speed_min)/(speed_max - speed_min) + offset, color="grey")
        else:  # flat line, should be simply offset
            axs[1].plot((speed_current_round - speed_min) + offset, color="grey")
        offset += 1.1
        # plot scaled calcium trace
        if temp_max != temp_min:
            axs[1].plot((temporal_current_round - temp_min)/(temp_max - temp_min) + offset, color="lightgreen")
        else:  # flat line
            axs[1].plot((temporal_current_round - temp_min) + offset, color="lightgreen")
        # plot all firing events that were recorded
        axs[1].vlines(x=firing_current_round, ymin=offset-0.05, ymax=offset+1.05, linewidth=1, color=[cmap.colors[i_color] for i_color in i_event_colors])
        offset += 1.1
    plt.tight_layout()
    plt.show()

In [ ]:
# TODO: plot ssm along with traces? (convert time frame index into spatial bin index, i.e. plot (i_frame, ssm_z[spatials[i_frame]]))
interact(plot_debug, i_cell=widgets.IntSlider(min=0, max=n_components-1, step=1, value=0))

### Calculate original and shuffled mean place coding vector lengths

In [ ]:
angles_on_belt = np.linspace(0, 2*np.pi, n_bins, endpoint=False) #initializing the circle with 150 bins corresponding to the distance on the belt
assert len(angles_on_belt) == n_bins

### Tuning vector analysis
Loosely based on Danielson et al. 2016: Distinct Contribution of Adult-Born Hippocampal Granule Cells to Context Encoding.
To calculate a tuning vector when spatial bins for one recording are given:
1. Get the location of each bin marked as having an event (`ssm_events_mask[i_cell] == 1`).
2. Get vector sum where each vector has unit length. 
3. Divide by number of vectors.

The algorithm:
1. Calculate original tuning vector for single cell
2. Shuffle bins within each round, find event flags, get tuning vector with new bins and flags.
3. Check the ratio of (# shuffled vectors shorter than original vector)/(# shuffles). This is the p value.


In [ ]:
ssm_events_mask.shape

In [ ]:
# cells not in idx_active_units will have np.nan, everything else overwritten
p_values_tuned = np.full(n_units, np.nan)  # p=1 is insignificance, suitable for cells which are not even included in analysis (not enough events). np.zeros would mark them as extremely place coding
shuffled_tuned_vector_lengths = np.full((n_units, analysis_params.n_shuffle), np.nan)
tv_lengths = np.full(n_units, np.nan)
tv_angles = np.full(n_units, np.nan)  # radians, 0 to 2pi range

for i_cell in idx_active_units:  # only use units with enough events
    ssm_cell = ssm_z[i_cell]
    ssm_events_cell = ssm_events_mask[i_cell]
    # add up the bins with corresponding angles. Ignore 0-length vectors for easier normalization with number of vectors
    i_rounds, i_bins = np.where(ssm_events_cell > 0)  # first array in tuple gives back row, second the column indices
    event_angles = angles_on_belt[i_bins]
    event_radii = ssm_cell[i_rounds, i_bins]
    tv_length, tv_angle = vector_sum(event_radii, event_angles )
    n_vectors = len(event_radii)  
    tv_length = tv_length/n_vectors  # normalize by number of vectors that were added up
    tv_lengths[i_cell] = tv_length
    tv_angles[i_cell] = tv_angle
    # Tuned vector length and angle for shuffled data
    tv_angles_shuffled = np.zeros(analysis_params.n_shuffle, dtype=np.float64)
    tv_lengths_shuffled = np.zeros(analysis_params.n_shuffle, dtype=np.float64)
    for i_shuffle in range(analysis_params.n_shuffle):
        #shuffle each row separately
        ssm_shuffled = ssm_cell.copy()  # Make a copy to avoid modifying original data
        for spiking_current_round in ssm_shuffled:
            np.random.shuffle(spiking_current_round)  # Shuffle the spiking events within each round independently
        # find flags again. 
        ssm_events_cell_shuffled = make_binary(ssm_shuffled, peak_threshold=analysis_params.peak_threshold,peak_distance=analysis_params.peak_distance)
        # Calculate mean direction and magnitude
        i_rounds_shuffled, i_bins_shuffled = np.where(ssm_events_cell_shuffled > 0)  # first array in tuple gives back row, second the column indices
        event_angles_shuffled = angles_on_belt[i_bins_shuffled]
        event_radii_shuffled = ssm_shuffled[i_rounds_shuffled, i_bins_shuffled]
        shuffled_length, shuffled_angle = vector_sum(event_radii_shuffled, event_angles_shuffled)    
        tv_lengths_shuffled[i_shuffle] = shuffled_length/n_vectors  # normalize by number of vectors that was added up
        tv_angles_shuffled[i_shuffle] = shuffled_angle
    shuffled_tuned_vector_lengths[i_cell] = tv_lengths_shuffled
    # calculate p-value as the percentile in which the candidate cell lies
    # large p_value (>0.95) signifies place coding
    p_value = float(np.sum(tv_lengths_shuffled >= tv_length)) / float(analysis_params.n_shuffle)
    p_values_tuned[i_cell] = p_value

### Kolmogorov-Smirnov test
Compare the measured data to a random shuffle (baseline). Then compare n_shuffle shuffles to the same baseline.
1. Randomly shift the spatial bins (Z-score) of each round to get a shuffled ssm. Then apply KS test to the original and shuffled ("baseline") bins, both averaged over rounds. This is the baseline statistic.
2. Shuffle the original data 1000 times, each time randomly shifting the spatial bins of each round. Compare the averaged bins of this shuffle and the baseline bins with the KS test.
3. Calculate the p-value as the ratio of (# [KS statistic of shuffled] > [KS statistic of baseline])/(# shuffles)

In [ ]:
p_values_ks = np.full(n_units, np.nan)  # p=1 is insignificance, suitable for cells which are not even included in analysis (not enough events). np.zeros would mark them as extremely place coding
shuffled_data_ks = np.full((n_units, analysis_params.n_shuffle), np.nan)
shuffled_bl_ks = np.full(n_units, np.nan)
if perform_ks:
    for i_cell in idx_active_units:
        ssm_cell = ssm_z[i_cell]
        # acquire baseline
        cell_spiking_bl = np.zeros(ssm_cell.shape)
        for i_round in range(len(cell_spiking_bl)):
                shift = random.randint(0, analysis_params.n_bins)
                cell_spiking_bl[i_round] = np.roll(ssm_cell[i_round], shift)  # Shuffle the spiking events within each round independently
        experiment_avg = np.mean(ssm_cell, axis=0)
        baseline_avg=np.mean(cell_spiking_bl,axis=0)
        ks_baseline,_=kstest(experiment_avg,baseline_avg)
        shuffled_bl_ks[i_cell] = ks_baseline
        # shuffle
        ks_shuffled = np.zeros(analysis_params.n_shuffle)
        for i_shuffle in range(analysis_params.n_shuffle):
            ssm_shuffled = np.zeros(ssm_cell.shape)
            for i_round in range(len(ssm_shuffled)):
                shift = random.randint(0, analysis_params.n_bins)  # TODO: shift lower range should be 1 or 0?
                ssm_shuffled[i_round] = np.roll(ssm_cell[i_round], shift)  # Shuffle the spiking events within each round independently
            shuffled_avg = np.mean(ssm_shuffled, axis=0)
            ks_shuffle,p_value_=kstest(baseline_avg, shuffled_avg)
            ks_shuffled[i_shuffle] = ks_shuffle
        shuffled_data_ks[i_cell] = ks_shuffled
        # calculate p value
        p_value_ks = np.sum(ks_shuffled >= ks_baseline) / len(ks_shuffled)
        p_values_ks[i_cell] = p_value_ks

# Plotting

In [ ]:
# TODO: make an ultimate debug plot:
# plot the trace per each round, and underline the frames that are included for analysis (when I do cuts, also cut for the frame indices in the end).
# Plot locomotion as well, to see that only frames with locomotion are included. Also plot the binary spikes. Then plot the spatial bins too? 
# As vlines or different colored spikes for each spatial bin?

## Load data into dataframes

### Signal strength per bin
Columns:  cell no., bin index, avg Signal strength over rounds, p_tuning, p_ks

Signal strength per cell per round per bin

In [ ]:
ssm_z_active = ssm_z[idx_active_units]
ssm_z_active_avg = np.mean(ssm_z_active, axis=1)  # average across rounds

In [ ]:
ssm_z_active.shape

In [ ]:
n_entries = ssm_z_active.flatten().shape[0]
col_cell_ids = np.zeros(ssm_z_active.shape, dtype=np.int16)
col_rounds = np.zeros(ssm_z_active.shape, dtype=np.int16)
col_bin_idxs = np.zeros(ssm_z_active.shape, dtype=np.int16)
# TODO loop over cells, rounds, and bins, add cell_id, round, bin_idx, firing_rate
for i_cell in range(ssm_z_active.shape[0]):
    col_cell_ids[i_cell] = np.full((ssm_z_active.shape[1], ssm_z_active.shape[2]), idx_active_units[i_cell], dtype=np.int16)
    for i_round in range(ssm_z.shape[1]):
        col_rounds[i_cell][i_round] = np.full(ssm_z.shape[2], i_round, dtype=np.int16)
        col_bin_idxs[i_cell][i_round] = np.linspace(0, ssm_z_avg.shape[1], num=ssm_z_avg.shape[1], endpoint=False, dtype=np.int16)
dict_fr={"cell_id":col_cell_ids.flatten(), "round":col_rounds.flatten(), "bin_idx":col_bin_idxs.flatten(), "firing_rate":ssm_z_active.flatten()}
df_firing_rate = pd.DataFrame(dict_fr)

Averaged signal strength over rounds

In [ ]:
n_entries = ssm_z_avg.flatten().shape[0]
col_cell_ids_avg = np.zeros(ssm_z_active_avg.shape, dtype=np.int16)
col_bin_idxs_avg = np.zeros(ssm_z_active_avg.shape, dtype=np.int16)
# TODO loop over cells, rounds, and bins, add cell_id, round, bin_idx, firing_rate
for i_cell in range(ssm_z_active_avg.shape[0]):
    col_cell_ids_avg[i_cell] = np.full(ssm_z_active_avg.shape[1], idx_active_units[i_cell])
    col_bin_idxs_avg[i_cell] = np.linspace(0, ssm_z_active_avg.shape[1], num=ssm_z_active_avg.shape[1], endpoint=False, dtype=np.int16)
dict_fr={"cell_id":col_cell_ids_avg.flatten(), "bin_idx":col_bin_idxs_avg.flatten(), "firing_rate":ssm_z_active_avg.flatten()}
df_avg_firing_rate = pd.DataFrame(dict_fr)

### P values per cell for both methods and bin index for first maximum of signal strength

In [ ]:
dict_p_vals = {"cell_id": np.array([i for i in range(n_units)]), "i_fr_max": np.argmax(ssm_z_avg, axis=1), "p_tuning": p_values_tuned, "p_ks": p_values_ks}
df_p_values = pd.DataFrame(data=dict_p_vals)
del dict_p_vals


### Join the dataframes

In [ ]:
df_avg_firing_rate = df_avg_firing_rate.join(df_p_values, how="left", on="cell_id", rsuffix="_other")

## Plot signal strength over belt for place cells vs non-place cells

### Tuned vector method

In [ ]:
pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_tuning"] <= 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
pivot_firing_rate_rejected = df_avg_firing_rate[df_avg_firing_rate["p_tuning"] > 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)

f, axs = plt.subplots(1, 2, figsize=(18, 6))
sns.heatmap(pivot_firing_rate_accepted, ax=axs[0])
sns.heatmap(pivot_firing_rate_rejected, ax=axs[1])

axs[0].set_title("accepted")
axs[1].set_title("rejected")
plt.show()

### KS method

In [ ]:
if perform_ks:
    pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_ks"] <= 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
    pivot_firing_rate_rejected = df_avg_firing_rate[df_avg_firing_rate["p_ks"] > 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)

    f, axs = plt.subplots(1, 2, figsize=(18, 6))
    sns.heatmap(pivot_firing_rate_accepted, ax=axs[0])
    sns.heatmap(pivot_firing_rate_rejected, ax=axs[1])
    axs[0].set_title("accepted")
    axs[1].set_title("rejected")

    plt.show()

## Cell-specific figures

In [ ]:
plot_single_cell = False
i_cell = 10

### Activity per round

In [ ]:
if plot_single_cell:
    #pivot_firing_rate_accepted = df_avg_firing_rate[df_avg_firing_rate["p_ks"] > 0.05].join(df_p_values, on="cell_id", how="left", rsuffix="_other").sort_values(by=["i_fr_max", "bin_idx"]).pivot_table(index="cell_id", columns="bin_idx", values="firing_rate", sort=False)
    pivot_table_single = df_firing_rate[df_firing_rate["cell_id"] == i_cell].pivot_table(columns="bin_idx", index="round", values="firing_rate")
    f, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(pivot_table_single, ax=ax)
    ax.set_title(f"Cell #{i_cell}")

    plt.show()

### Shuffle histograms

In [ ]:
if plot_single_cell:
    fig, axs = plt.subplots(1,2, figsize=(12, 4))
    axs[0].hist(shuffled_tuned_vector_lengths[i_cell])
    axs[1].hist(shuffled_data_ks[i_cell])

    axs[0].vlines(x=tv_lengths[i_cell], ymin=0, ymax=100, color="red")
    axs[1].vlines(x=shuffled_bl_ks[i_cell], ymin=0, ymax=100, color="red")

    plt.show()

### Circular plotting of activity

In [ ]:
if plot_single_cell:
    i_cell = 359
    binary_spiking_cell = ssm_events_mask[i_cell]
    n_bins = len(binary_spiking_cell[0])
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.set_xlabel('Angle (degrees)')
    ax.set_ylabel('Radius')
    for i_round in range(len(binary_spiking_cell)):
        i_firing_bins = binary_spiking_cell[i_round].nonzero()[0]
        if len(i_firing_bins) > 0:
            tv_length = i_round + 1  # avoid 0 length
            vector_lengths = [tv_length]*len(i_firing_bins)
            firing_angles_deg = i_firing_bins*2*math.pi/n_bins
            ax.scatter(firing_angles_deg, vector_lengths)
    plt.show()

# Interactive plotting

In [ ]:
def plot_data(i_active_cell):
    i_cell = idx_active_units[i_active_cell]
    pivot_table_single = df_firing_rate[df_firing_rate["cell_id"] == i_cell].pivot_table(columns="bin_idx", index="round", values="firing_rate")
    fig = plt.figure(figsize=(12, 12))
    binary_spiking_cell = ssm_events_mask[i_cell]
    n_bins = len(binary_spiking_cell[0])
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(222, projection='polar')
    ax3 = plt.subplot(223)
    ax4  =plt.subplot(224)
    sns.heatmap(pivot_table_single, ax=ax1)
    ax2.set_xlabel('Angle (degrees)')
    ax2.set_ylabel('Round')
    vector_lengths, i_bins = np.where(binary_spiking_cell)  # vector lengths will be round index, starting with 1
    vector_lengths += 1
    firing_angles_rad = i_bins*2*np.pi/n_bins
    ax2.scatter(firing_angles_rad, vector_lengths)
    ax2.set_ylim((0, len(binary_spiking_cell)+0.2))
    ax1.set_title(f"Cell #{i_cell}")
    ax3.set_title(f"Tuning vector length vs shuffled data, p={p_values_tuned[i_cell]}")
    ax3.hist(shuffled_tuned_vector_lengths[i_cell], bins=20)
    ax3.vlines(x=tv_lengths[i_cell], ymin=0, ymax=100, color="red")
    if perform_ks:
        ax4.hist(shuffled_data_ks[i_cell], bins=20)
        ax4.vlines(x=shuffled_bl_ks[i_cell], ymin=0, ymax=100, color="red")
        ax4.set_title(f"K-S test, p={p_values_ks[i_cell]}")

    plt.show()

In [ ]:
np.corrcoef(p_values_tuned, p_values_ks)
  

In [ ]:
interact(plot_data, i_active_cell=widgets.IntSlider(min=0, max=len(idx_active_units)-1, step=1, value=0))

# Export results

In [ ]:
fpath_output = os.path.join(output_folder, os.path.splitext(os.path.split(fpath_expinfo)[-1])[0].replace("expinfo", "placecoding")+".h5")
print(f"Saving to {fpath_output}")

In [ ]:
if save_results:
    with h5py.File(fpath_output, "w") as hf:
        dict_analysis_params = analysis_params.to_dict()
        for key in dict_analysis_params.keys():
            if dict_analysis_params[key] is not None:
                hf.attrs[key] = dict_analysis_params[key]
            else:
                hf.attrs[key] = np.nan
        dict_exp_info = exp_info.to_dict()
        for key in dict_exp_info.keys():
            if dict_exp_info[key] is not None:
                hf.attrs[key] = dict_exp_info[key]
            else:
                hf.attrs[key] = np.nan
        # temporal and spatial components
        hf.create_dataset("temporal_raw", data=temporal_raw)
        hf.create_dataset("A_data", data=A_data)
        hf.create_dataset("A_indices", data=A_indices)
        hf.create_dataset("A_indptr", data=A_indptr)
        hf.create_dataset("A_shape", data=A_shape)
        hf.attrs["resolution"] = resolution

        hf.create_dataset("ssm_raw", data=ssm_raw)
        hf.create_dataset("ssm_z", data=ssm_z)
        hf.create_dataset("ssm_events_mask", data=ssm_events_mask)
        hf.create_dataset("tuned_vector_lengths", data=tv_lengths)
        hf.create_dataset("tuned_vector_angles", data=tv_angles)
        hf.create_dataset("shuffled_tuned_vector_lengths", data=shuffled_tuned_vector_lengths)
        hf.create_dataset("p_values_tuned", data=p_values_tuned)
        hf.create_dataset("shuffled_data_ks", data=shuffled_data_ks)
        hf.create_dataset("shuffled_bl_ks", data=shuffled_bl_ks)
        hf.create_dataset("p_values_ks", data=p_values_ks)
        hf.create_dataset("idx_active_units", data=idx_active_units)

        # TODO: add spatial components (to make it a standalone file) as sparse matrix
    print(f"Saved to {fpath_output}")
else:
    print("Not saved.")
